<a href="https://colab.research.google.com/github/Denis2054/Context-Engineering-for-Multi-Agent-Systems/blob/main/Chapter09/Marketing_Assistant_Token_0ptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marketing Assistant

Copyright 2025-2026, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases

*Note* Make sure to run `Data_Ingestion_Marketing.ipynb` first to ingest the data for this notebook.


# I. Inititalization

## GitHub

In [ ]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   5265      0 --:--:-- --:--:-- --:--:--  5268
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  12343      0 --:--:-- --:--:-- --:--:-- 12340
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  26279      0 --:--:-- --:--:-- --:--:-- 26295
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  10536      0 --:--:-- --:--:-- --:--:-- 10558
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Render and Trace Dashboard

In [ ]:
import json
import html
import markdown
from IPython.display import display, HTML

def render_trace_dashboard(trace):
    """
    Generates a clean HTML dashboard for the Context Engine Execution Trace.
    UPGRADE: Now includes token metrics (In, Out, Saved) for each step.
    """
    # Define CSS styles for the dashboard
    css = """
    <style>
        :root {
            --primary-color: #2563eb;
            --success-color: #22c55e;
            --error-color: #ef4444;
            --bg-color: #f8fafc;
            --card-bg: #ffffff;
            --text-main: #1e293b;
            --text-muted: #64748b;
            --border-color: #e2e8f0;
            --metric-bg: #f1f5f9;
        }
        .dashboard-container {
            font-family: 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
            background-color: var(--bg-color);
            border: 1px solid var(--border-color);
            border-radius: 12px;
            padding: 24px;
            max-width: 100%;
            box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
            margin-top: 20px;
        }
        .header-section {
            border-bottom: 2px solid var(--border-color);
            padding-bottom: 16px;
            margin-bottom: 20px;
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
        }
        .header-title { margin: 0; font-size: 1.5rem; color: var(--text-main); font-weight: 700; }
        .header-goal { margin: 8px 0 0 0; color: var(--text-muted); font-size: 1rem; font-style: italic;}
        .status-badge {
            padding: 6px 12px;
            border-radius: 20px;
            font-weight: 600;
            font-size: 0.875rem;
            color: white;
            white-space: nowrap;
        }
        .status-success { background-color: var(--success-color); }
        .status-failure { background-color: var(--error-color); }

        /* Metrics Bar Styling */
        .metrics-bar {
            display: flex;
            gap: 12px;
            margin-top: 8px;
            font-size: 0.75rem;
            font-weight: 600;
        }
        .metric-pill {
            background-color: var(--metric-bg);
            color: var(--text-muted);
            padding: 2px 8px;
            border-radius: 4px;
            border: 1px solid var(--border-color);
        }
        .metric-saved {
            background-color: #dcfce7;
            color: #166534;
            border-color: #bbf7d0;
        }

        /* Step Cards */
        .step-card {
            background-color: var(--card-bg);
            border: 1px solid var(--border-color);
            border-radius: 8px;
            margin-bottom: 16px;
            overflow: hidden;
        }
        summary.step-header {
            padding: 16px;
            background-color: #f8fafc;
            cursor: pointer;
            list-style: none;
            display: flex;
            align-items: center;
            justify-content: space-between;
        }
        .agent-badge {
            background-color: var(--primary-color);
            color: white;
            padding: 4px 10px;
            border-radius: 6px;
            font-size: 0.75rem;
            text-transform: uppercase;
            margin-left: 10px;
        }
        .step-content { padding: 16px; border-top: 1px solid var(--border-color); }
        .data-label {
            font-size: 0.75rem;
            text-transform: uppercase;
            color: var(--text-muted);
            font-weight: 700;
            margin-bottom: 6px;
        }
        .json-box {
            background-color: #1e1e1e;
            color: #d4d4d4;
            padding: 12px;
            border-radius: 6px;
            font-family: monospace;
            font-size: 0.85rem;
            overflow-x: auto;
            white-space: pre-wrap;
        }
        .rendered-content {
            background-color: #fff;
            border: 1px solid #e2e8f0;
            border-left: 4px solid var(--primary-color);
            border-radius: 4px;
            padding: 12px 16px;
        }
        .final-output-card {
            border: 2px solid var(--success-color);
            background-color: #f0fdf4;
            border-radius: 8px;
            padding: 20px;
            margin-top: 30px;
        }
    </style>
    """

    status_class = "status-success" if trace.status == "Success" else "status-failure"

    dashboard_html = f"""
    {css}
    <div class="dashboard-container">
        <div class="header-section">
            <div>
                <h1 class="header-title">Context Engine Trace</h1>
                <p class="header-goal">"{html.escape(trace.goal)}"</p>
            </div>
            <div style="text-align: right;">
                <span class="status-badge {status_class}">{trace.status}</span>
                <div style="margin-top: 5px; font-size: 0.85rem; color: #64748b;">
                    Time: {trace.duration:.2f}s
                </div>
            </div>
        </div>

        <div class="steps-container">
            <h3 style="color: var(--text-main); margin-bottom: 15px;">Execution Steps</h3>
    """

    for step in trace.steps:
        # Resolve Input JSON
        try:
            resolved_ctx = json.dumps(step['resolved_context'], indent=2)
        except:
            resolved_ctx = str(step['resolved_context'])

        # Handle Output Rendering
        output_raw = step['output']
        rendered_html = ""

        def render_md(text):
            return markdown.markdown(text) if text else ""

        if isinstance(output_raw, dict):
            for key in ['summary', 'answer_with_sources', 'answer', 'output', 'content']:
                if key in output_raw and isinstance(output_raw[key], str):
                    content = output_raw[key]
                    rendered_html = f'<div class="rendered-content">{render_md(content)}</div>'
                    break
            raw_json_display = json.dumps(output_raw, indent=2)
        else:
            rendered_html = f'<div class="rendered-content">{render_md(str(output_raw))}</div>'
            raw_json_display = str(output_raw)

        # UPGRADE: Build the Metrics Display for this step
        t_in = step.get('tokens_in', 0)
        t_out = step.get('tokens_out', 0)
        t_saved = step.get('tokens_saved', 0)

        metrics_html = f"""
        <div class="metrics-bar">
            <span class="metric-pill">📥 In: {t_in}</span>
            <span class="metric-pill">📤 Out: {t_out}</span>
        """
        if t_saved > 0:
            metrics_html += f'<span class="metric-pill metric-saved">📉 Saved: {t_saved}</span>'
        metrics_html += "</div>"

        # Build Step Card
        step_html = f"""
            <details class="step-card" open>
                <summary class="step-header">
                    <div>
                        <span style="font-weight:700;">Step {step['step']}</span>
                        <span class="agent-badge">{step['agent']}</span>
                        {metrics_html}
                    </div>
                    <span style="font-size: 0.8rem; color: #94a3b8;">▼ Details</span>
                </summary>
                <div class="step-content">
                    <div style="margin-bottom:15px;">
                        <div class="data-label">Resolved Input</div>
                        <details><summary style="font-size:0.7rem; cursor:pointer;">View JSON</summary>
                        <div class="json-box">{html.escape(resolved_ctx)}</div></details>
                    </div>
                    <div>
                        <div class="data-label">Agent Output</div>
                        {rendered_html}
                    </div>
                </div>
            </details>
        """
        dashboard_html += step_html

    # Final Result
    if trace.final_output:
        dashboard_html += f"""
        <div class="final-output-card">
            <div style="color:var(--success-color); font-weight:bold; margin-bottom:10px;">FINAL RESULT</div>
            {render_md(str(trace.final_output))}
        </div>
        """

    dashboard_html += "</div>"
    display(HTML(dashboard_html))

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# === ENGINE ROOM: The Main Execution Function (Visualizer Update) ===

import logging
import pprint
import json
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine with HTML dashboard visualization.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---\n
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            pprint.pprint(moderation_report)
            return

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---\n
    if result and moderation_active:
        # Flatten result for checking
        text_to_check = str(result)
        if isinstance(result, (dict, list)):
            text_to_check = json.dumps(result)

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"
            # We also scrub the trace result to prevent display of harmful content
            trace.final_output = result

    # 2. Render the HTML Dashboard
    # This replaces the raw text printouts with the visualizer
    if trace:
        render_trace_dashboard(trace)
    else:
        print("Engine failed to initialize trace.")

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---


In [ ]:
#@title Product Marketing Copy Generation(Use Case 1)
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

In [ ]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

In [ ]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)